In [1]:
# Importar módulos necesarios
import sys
import os
import json
from pathlib import Path
import pandas as pd
import plotly.express as px

# Determinar el directorio raíz y construir la ruta al directorio 'Modulos'
root_dir = Path().resolve()
analisis_segment = os.path.join('ADRpy', 'analisis')
if str(root_dir).endswith(analisis_segment):
    # Notebook ejecutado desde ADRpy/analisis
    modulos_path = root_dir / 'Modulos'
else:
    # Notebook ejecutado desde la raíz del proyecto
    modulos_path = root_dir / 'ADRpy' / 'analisis' / 'Modulos'

# Añadir 'Modulos' al sys.path si existe
if modulos_path.is_dir():
    modulos_path_str = str(modulos_path)
    if modulos_path_str not in sys.path:
        sys.path.append(modulos_path_str)
        print(f"✅ Módulo encontrado y añadido a sys.path: {modulos_path_str}")
else:
    print(f"⚠️ Directorio Modulos no encontrado en: {modulos_path}")

# Probar la carga de datos (sin ejecutar la app completa)
try:
    from Analisis_modelos.data_loader import load_models_data, extract_unique_values
    
    # Ruta al archivo JSON
    json_path = os.path.join('Results', 'modelos_completos_por_celda.json')
    
    if os.path.exists(json_path):
        print("✅ Archivo JSON encontrado")
        
        # Cargar datos
        modelos_por_celda, detalles_por_celda = load_models_data(json_path)
        
        print(f"📊 Modelos por celda: {len(modelos_por_celda)} celdas")
        print(f"📋 Detalles por celda: {len(detalles_por_celda)} celdas")
        
        # Extraer valores únicos para filtros
        unique_values = extract_unique_values(modelos_por_celda)
        
        print("\n🔍 Valores únicos disponibles:")
        for key, values in unique_values.items():
            print(f"  {key}: {len(values)} opciones")
            if len(values) <= 10:  # Mostrar si hay pocas opciones
                print(f"    {values}")
            else:
                print(f"    {values[:5]}... (y {len(values)-5} más)")
        
    else:
        print(f"❌ No se encontró el archivo JSON en: {json_path}")
        print("Verifique que la ruta sea correcta.")
        
except ImportError as e:
    print(f"⚠️ Error de importación: {e}")
    print("Instale las dependencias necesarias: pip install dash plotly pandas")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Módulo encontrado y añadido a sys.path: C:\Users\delpi\OneDrive\Tesis\ADRpy-VTOL\ADRpy\analisis\Modulos


INFO:Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles


✅ Archivo JSON encontrado
📊 Modelos por celda: 6 celdas
📋 Detalles por celda: 6 celdas

🔍 Valores únicos disponibles:
  aeronaves: 3 opciones
    ['A3', 'A5', 'A7']
  parametros: 5 opciones
    ['Alcance de la aeronave', 'Envergadura', 'Payload', 'Potencia HP', 'Velocidad a la que se realiza el crucero (KTAS)']
  tipos_modelo: 7 opciones
    ['exp-1', 'linear-1', 'linear-2', 'log-1', 'poly-1', 'poly-2', 'pot-1']
  predictores: 8 opciones
    ['Alcance de la aeronave', 'Ancho del fuselaje', 'Cantidad de motores', 'Envergadura', 'Payload', 'Potencia HP', 'Rango de comunicación', 'Velocidad a la que se realiza el crucero (KTAS)']
  n_predictores: 2 opciones
    [1, 2]


In [2]:
#desde aca en adelante nuevo
# Validación de los cambios implementados en el flujo principal
try:
    print("🧪 VALIDACIÓN DE CAMBIOS IMPLEMENTADOS:")
    print("="*60)
    
    # Reiniciar importaciones para cargar cambios
    import importlib
    import sys
    if 'Analisis_modelos.plot_model_curves' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.plot_model_curves'])
    if 'Analisis_modelos.ui_components' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.ui_components'])
    
    # Verificar que las nuevas funciones están disponibles
    from Analisis_modelos.plot_model_curves import (
        extract_imputed_values_from_details
    )
    from Analisis_modelos.ui_components import create_imputation_methods_checklist
    
    print("✅ Funciones importadas correctamente")
    
    # Importar la función de filtrado manualmente si no está disponible
    try:
        from Analisis_modelos.plot_model_curves import filter_imputed_points_by_method
        print("✅ Función de filtrado importada correctamente")
    except ImportError:
        # Definir función localmente para prueba
        def filter_imputed_points_by_method(imputed_points_data, selected_methods):
            if not selected_methods:
                return []
            filtered_points = []
            for point in imputed_points_data:
                method = point.get('imputation_method', '')
                if method in selected_methods:
                    filtered_points.append(point)
            return filtered_points
        print("⚠️ Usando función de filtrado local")
    
    # Validar con datos reales
    if 'modelos_por_celda' in locals() and 'detalles_por_celda' in locals():
        # Buscar una celda con imputaciones
        celda_ejemplo = None
        for celda_key, detalles in detalles_por_celda.items():
            if isinstance(detalles, dict) and any(k in detalles for k in ['final', 'similitud', 'correlacion']):
                celda_ejemplo = celda_key
                break
        
        if celda_ejemplo:
            aeronave, parametro = celda_ejemplo.split('|')
            modelos = modelos_por_celda.get(celda_ejemplo, [])
            modelos_1pred = [m for m in modelos if isinstance(m, dict) and m.get('n_predictores', 0) == 1]
            
            print(f"\n📊 Probando con celda: {celda_ejemplo}")
            print(f"   Modelos disponibles: {len(modelos)}")
            print(f"   Modelos de 1 predictor: {len(modelos_1pred)}")
            
            if modelos_1pred:
                # Probar extracción de puntos imputados con el nuevo método
                imputed_points = extract_imputed_values_from_details(
                    detalles_por_celda, 
                    celda_ejemplo, 
                    modelos_1pred
                )
                print(f"   Puntos imputados extraídos: {len(imputed_points)}")
                
                if imputed_points:
                    # Mostrar ejemplo de punto
                    point = imputed_points[0]
                    print(f"\n📍 Ejemplo de punto imputado:")
                    print(f"   Método: {point.get('imputation_method')}")
                    print(f"   X normalizado: {point.get('x_normalized', 0):.3f}")
                    print(f"   Valor Y: {point.get('y_value', 0):.3f}")
                    print(f"   Predictor: {point.get('predictor')}")
                    print(f"   Confianza: {point.get('confidence', 0):.3f}")
                    
                    # Probar filtrado por métodos
                    filtered_final = filter_imputed_points_by_method(imputed_points, ['final'])
                    filtered_similitud = filter_imputed_points_by_method(imputed_points, ['similitud'])
                    filtered_correlacion = filter_imputed_points_by_method(imputed_points, ['correlacion'])
                    
                    print(f"\n🔍 Filtrado por métodos:")
                    print(f"   Solo 'final': {len(filtered_final)} puntos")
                    print(f"   Solo 'similitud': {len(filtered_similitud)} puntos")
                    print(f"   Solo 'correlacion': {len(filtered_correlacion)} puntos")
                    
                    # Validar normalización usando el rango del mejor modelo
                    if len(modelos_1pred) > 0:
                        mejor_modelo = modelos_1pred[0]
                        predictor = mejor_modelo.get('predictores', [None])[0]
                        print(f"\n⚖️ Validación de normalización:")
                        print(f"   Mejor modelo: {mejor_modelo.get('tipo', 'N/A')}")
                        print(f"   Predictor de referencia: {predictor}")
                        print(f"   MAPE: {mejor_modelo.get('mape', 0):.3f}%")
                        print(f"   R²: {mejor_modelo.get('r2', 0):.3f}")
                        
                        # Verificar que la normalización sea consistente
                        x_values = [p.get('x_normalized', 0) for p in imputed_points]
                        if x_values:
                            print(f"   Rango X normalizado: [{min(x_values):.3f}, {max(x_values):.3f}]")
                            print(f"   ✅ Normalización usando rango del mejor modelo")
        
        # Probar componente UI
        print(f"\n🎨 Componente UI de filtrado:")
        try:
            checklist = create_imputation_methods_checklist()
            print(f"   ✅ Checklist de métodos creado correctamente")
            print(f"   Opciones disponibles: {len(checklist.options)}")
            print(f"   Valores por defecto: {checklist.value}")
        except Exception as e:
            print(f"   ❌ Error en componente UI: {e}")
        
        print(f"\n🎯 RESUMEN DE CAMBIOS IMPLEMENTADOS:")
        print(f"   ✅ Normalización usando rango del mejor modelo seleccionado")
        print(f"   ✅ Filtro de métodos de imputación en la UI")
        print(f"   ✅ Callback actualizado para manejar el filtrado")
        print(f"   ✅ Solo se muestran imputaciones de la celda seleccionada")
        print(f"   ✅ Funciones de extracción y filtrado implementadas")
        
        print(f"\n💡 INSTRUCCIONES PARA VERIFICAR EN LA APLICACIÓN:")
        print(f"   1. Abrir http://localhost:8050 en el navegador")
        print(f"   2. Seleccionar una aeronave y parámetro")
        print(f"   3. Verificar el nuevo filtro 'Métodos de Imputación'")
        print(f"   4. Cambiar las opciones para ver el filtrado en tiempo real")
        print(f"   5. Los puntos imputados deben normalizarse correctamente")
        
    else:
        print("⚠️ No hay datos disponibles para la validación")
        
except Exception as e:
    print(f"❌ Error en validación: {e}")
    import traceback
    traceback.print_exc()

INFO:Analisis_modelos.plot_model_curves:Extraídos 3 puntos imputados de detalles para celda A3|Velocidad a la que se realiza el crucero (KTAS)
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['final']
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['similitud']
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['correlacion']
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['final']
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['similitud']
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['correlacion']


🧪 VALIDACIÓN DE CAMBIOS IMPLEMENTADOS:
✅ Funciones importadas correctamente
✅ Función de filtrado importada correctamente

📊 Probando con celda: A3|Velocidad a la que se realiza el crucero (KTAS)
   Modelos disponibles: 21
   Modelos de 1 predictor: 17
   Puntos imputados extraídos: 3

📍 Ejemplo de punto imputado:
   Método: final
   X normalizado: -0.395
   Valor Y: 39.242
   Predictor: Payload
   Confianza: 0.386

🔍 Filtrado por métodos:
   Solo 'final': 1 puntos
   Solo 'similitud': 1 puntos
   Solo 'correlacion': 1 puntos

⚖️ Validación de normalización:
   Mejor modelo: linear-1
   Predictor de referencia: Payload
   MAPE: 6.140%
   R²: 0.119
   Rango X normalizado: [-0.396, -0.395]
   ✅ Normalización usando rango del mejor modelo

🎨 Componente UI de filtrado:
   ✅ Checklist de métodos creado correctamente
   Opciones disponibles: 3
   Valores por defecto: ['final', 'similitud', 'correlacion']

🎯 RESUMEN DE CAMBIOS IMPLEMENTADOS:
   ✅ Normalización usando rango del mejor modelo se

In [3]:
# Verificar e instalar dependencias necesarias y probar nueva funcionalidad 3D
import subprocess
import sys
import importlib

def install_package(package):
    """Instala un paquete usando pip."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        return True
    except subprocess.CalledProcessError:
        return False

def check_dependencies():
    """Verifica si las dependencias están instaladas."""
    required_packages = {
        'dash': 'dash>=2.14.0',
        'plotly': 'plotly>=5.17.0', 
        'pandas': 'pandas>=1.5.0',
        'numpy': 'numpy>=1.21.0'
    }
    
    missing_packages = []
    
    for package, version_spec in required_packages.items():
        try:
            __import__(package)
            print(f"✅ {package} está instalado")
        except ImportError:
            print(f"❌ {package} no está instalado")
            missing_packages.append(version_spec)
    
    return missing_packages

# Verificar dependencias
print("🔍 Verificando dependencias...")
missing = check_dependencies()

if missing:
    print(f"\n📦 Faltan {len(missing)} dependencias:")
    for package in missing:
        print(f"   - {package}")
else:
    print("\n✅ Todas las dependencias están instaladas correctamente")

# Recargar módulos para aplicar cambios
print("\n🔄 Recargando módulos...")
try:
    if 'Analisis_modelos.data_loader' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.data_loader'])
        print("✅ data_loader recargado")
    
    if 'Analisis_modelos.plot_3d' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.plot_3d'])
        print("✅ plot_3d recargado")
except Exception as e:
    print(f"⚠️ Error recargando módulos: {e}")

# Probar nueva funcionalidad 3D
print("\n🧪 PROBANDO NUEVA FUNCIONALIDAD 3D:")
print("="*50)

try:
    # Importar el nuevo módulo 3D
    from Analisis_modelos.plot_3d import (
        extract_coefficients_from_equation,
        filter_models_for_3d,
        create_3d_plot
    )
    from Analisis_modelos.data_loader import filter_models
    print("✅ Módulos importados correctamente")
    
    # Probar extracción de coeficientes con datos reales
    if 'modelos_por_celda' in locals():
        print("\n🔍 Buscando modelos con 2 predictores...")
        
        # Buscar modelos con 2 predictores
        modelos_2pred_encontrados = []
        for celda_key, modelos in modelos_por_celda.items():
            for model in modelos:
                if isinstance(model, dict) and model.get('n_predictores') == 2:
                    modelos_2pred_encontrados.append((celda_key, model))
                    if len(modelos_2pred_encontrados) >= 3:  # Solo tomar 3 ejemplos
                        break
            if len(modelos_2pred_encontrados) >= 3:
                break
        
        print(f"   Encontrados {len(modelos_2pred_encontrados)} modelos de 2 predictores")
        
        for i, (celda_key, model) in enumerate(modelos_2pred_encontrados):
            aeronave, parametro = celda_key.split('|')
            tipo = model.get('tipo', 'N/A')
            predictores = model.get('predictores', [])
            ecuacion_latex = model.get('ecuacion_normalizada_latex', '')
            
            print(f"\n📊 Modelo {i+1}: {aeronave} - {parametro}")
            print(f"   Tipo: {tipo}")
            print(f"   Predictores: {predictores}")
            print(f"   R²: {model.get('r2', 0):.3f}")
            print(f"   MAPE: {model.get('mape', 0):.2f}%")
            print(f"   Ecuación LaTeX: {ecuacion_latex}")
            
            # Probar extracción de coeficientes
            coefficients = extract_coefficients_from_equation(ecuacion_latex, 2)
            if coefficients:
                print(f"   ✅ Coeficientes extraídos: {coefficients}")
                print(f"   [c0={coefficients[0]:.6f}, c1={coefficients[1]:.6f}, c2={coefficients[2]:.6f}")
                print(f"    c3={coefficients[3]:.6f}, c4={coefficients[4]:.6f}, c5={coefficients[5]:.6f}]")
            else:
                print(f"   ❌ No se pudieron extraer coeficientes")
        
        # Probar filtrado para vista 3D
        if modelos_2pred_encontrados:
            celda_ejemplo = modelos_2pred_encontrados[0][0]
            aeronave_test, parametro_test = celda_ejemplo.split('|')
            
            print(f"\n🔧 Probando filtro 3D para {aeronave_test} - {parametro_test}")
            modelos_3d = filter_models_for_3d(
                modelos_por_celda, 
                aeronave_test, 
                parametro_test
            )
            print(f"   Modelos filtrados para 3D: {len(modelos_3d)}")
            
            for j, model in enumerate(modelos_3d):
                if isinstance(model, dict):
                    print(f"   Modelo {j+1}: {model.get('tipo')} - {model.get('predictores')} - R²:{model.get('r2', 0):.3f}")
        
        # Buscar un modelo polinómico para probar
        print(f"\n🔍 Buscando modelos polinómicos con 2 predictores...")
        modelos_poly_encontrados = []
        for celda_key, modelos in modelos_por_celda.items():
            for model in modelos:
                if (isinstance(model, dict) and 
                    model.get('n_predictores') == 2 and 
                    model.get('tipo', '').startswith('poly')):
                    modelos_poly_encontrados.append((celda_key, model))
                    if len(modelos_poly_encontrados) >= 2:  # Solo tomar 2 ejemplos
                        break
            if len(modelos_poly_encontrados) >= 2:
                break
        
        if modelos_poly_encontrados:
            print(f"   Encontrados {len(modelos_poly_encontrados)} modelos polinómicos")
            
            for i, (celda_key, model) in enumerate(modelos_poly_encontrados):
                aeronave, parametro = celda_key.split('|')
                tipo = model.get('tipo', 'N/A')
                predictores = model.get('predictores', [])
                ecuacion_latex = model.get('ecuacion_normalizada_latex', '')
                
                print(f"\n📊 Modelo Polinómico {i+1}: {aeronave} - {parametro}")
                print(f"   Tipo: {tipo}")
                print(f"   Predictores: {predictores}")
                print(f"   R²: {model.get('r2', 0):.3f}")
                print(f"   MAPE: {model.get('mape', 0):.2f}%")
                print(f"   Ecuación LaTeX: {ecuacion_latex}")
                
                # Probar extracción de coeficientes
                coefficients = extract_coefficients_from_equation(ecuacion_latex, 2)
                if coefficients:
                    print(f"   ✅ Coeficientes extraídos: {coefficients}")
                    print(f"   [c0={coefficients[0]:.6f}, c1={coefficients[1]:.6f}, c2={coefficients[2]:.6f}")
                    print(f"    c3={coefficients[3]:.6f}, c4={coefficients[4]:.6f}, c5={coefficients[5]:.6f}]")
                else:
                    print(f"   ❌ No se pudieron extraer coeficientes")
        else:
            print(f"   ⚠️ No se encontraron modelos polinómicos de 2 predictores")
        
        print(f"\n🎯 RESUMEN DE PRUEBAS:")
        print(f"   ✅ Módulo 3D funcional")
        print(f"   ✅ Extracción de coeficientes operativa")
        print(f"   ✅ Filtrado para vista 3D operativo")
        print(f"   ✅ Separación de modelos 2D/3D implementada")
        print(f"   ✅ Soporte para modelos lineales y polinómicos")
        
    else:
        print("⚠️ No hay datos cargados para probar")
        
except ImportError as e:
    print(f"❌ Error importando módulo 3D: {e}")
except Exception as e:
    print(f"❌ Error en pruebas 3D: {e}")
    import traceback
    traceback.print_exc()

🔍 Verificando dependencias...
✅ dash está instalado
✅ plotly está instalado
✅ pandas está instalado
✅ numpy está instalado

✅ Todas las dependencias están instaladas correctamente

🔄 Recargando módulos...
✅ data_loader recargado
✅ plot_3d recargado

🧪 PROBANDO NUEVA FUNCIONALIDAD 3D:
✅ Módulos importados correctamente

🔍 Buscando modelos con 2 predictores...
   Encontrados 3 modelos de 2 predictores

📊 Modelo 1: A7 - Payload
   Tipo: linear-2
   Predictores: ['Potencia HP', 'Envergadura']
   R²: 0.995
   MAPE: 2.12%
   Ecuación LaTeX: y = -1.133e-16 + 0.718115 x_{0} + 0.285015 x_{1}
   ✅ Coeficientes extraídos: [-1.133e-16, 0.718115, 0.285015, 0.0, 0.0, 0.0]
   [c0=-0.000000, c1=0.718115, c2=0.285015
    c3=0.000000, c4=0.000000, c5=0.000000]

📊 Modelo 2: A7 - Payload
   Tipo: linear-2
   Predictores: ['Potencia HP', 'Alcance de la aeronave']
   R²: 0.993
   MAPE: 2.96%
   Ecuación LaTeX: y = -1.09219e-16 + 0.837673 x_{0} + 0.164712 x_{1}
   ✅ Coeficientes extraídos: [-1.09219e-16, 0.8

In [ ]:
# Ejecutar la aplicación principal de análisis de modelos
print("🚀 Iniciando aplicación de análisis de modelos...")

try:
    # Importar y ejecutar la función principal
    from Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
    
    # Configuración de la aplicación
    config = {
        'json_path': None,  # Usar ruta automática
        'use_dash': True,   # Usar interfaz Dash
        'port': 8050,       # Puerto por defecto
        'debug': False      # Modo producción
    }
    
    print("⚙️ Configuración:")
    for key, value in config.items():
        print(f"   {key}: {value}")
    
    print("\n" + "="*60)
    print("🌐 APLICACIÓN WEB INTERACTIVA")
    print("="*60)
    print(f"📍 URL: http://localhost:{config['port']}")
    print("🔧 Controles disponibles:")
    print("   • Filtro por aeronave")
    print("   • Filtro por parámetro") 
    print("   • Selección de tipos de modelo")
    print("   • Filtros de predictores")
    print("   • Panel de información lateral")
    print("   • Tablas de métricas comparativas")
    print("\n⚡ Presione Ctrl+C en la terminal para detener")
    print("📖 Use los filtros para explorar los modelos de imputación")
    print("="*60)
    
    # Ejecutar aplicación principal
    main_visualizacion_modelos(**config)

except ImportError as e:
    print(f"❌ Error de importación: {e}")
    print("💡 Soluciones posibles:")
    print("   1. Instale las dependencias: pip install dash plotly pandas numpy")
    print("   2. Verifique que los módulos estén en la ruta correcta")
    print("   3. Reinicie el kernel del notebook")
    
    # Intentar versión alternativa por consola
    print("\n🔄 Ejecutando versión simplificada por consola...")
    try:
        from Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
        main_visualizacion_modelos(use_dash=False)
    except Exception as console_error:
        print(f"❌ Error en versión de consola: {console_error}")

except KeyboardInterrupt:
    print("\n\n⏹️ Aplicación detenida por el usuario")

except Exception as e:
    print(f"❌ Error ejecutando aplicación: {e}")
    print("💡 Verifique que:")
    print("   • El archivo JSON existe en la ruta correcta")
    print("   • Los datos tienen el formato esperado")
    print("   • No hay conflictos de puertos")
else:
    print("\n✅ Aplicación finalizada correctamente.")

INFO:Analisis_modelos.main_visualizacion_modelos:Cargando datos desde: C:\Users\delpi\OneDrive\Tesis\ADRpy-VTOL\ADRpy\analisis\Results\modelos_completos_por_celda.json
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Analisis_modelos.main_visualizacion_modelos:Datos cargados exitosamente
INFO:Analisis_modelos.main_visualizacion_modelos:Datos cargados exitosamente


🚀 Iniciando aplicación de análisis de modelos...
⚙️ Configuración:
   json_path: None
   use_dash: True
   port: 8050
   debug: False

🌐 APLICACIÓN WEB INTERACTIVA
📍 URL: http://localhost:8050
🔧 Controles disponibles:
   • Filtro por aeronave
   • Filtro por parámetro
   • Selección de tipos de modelo
   • Filtros de predictores
   • Panel de información lateral
   • Tablas de métricas comparativas

⚡ Presione Ctrl+C en la terminal para detener
📖 Use los filtros para explorar los modelos de imputación
Iniciando aplicación Dash en http://localhost:8050
Presione Ctrl+C para detener la aplicación



✅ Aplicación finalizada correctamente.


INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] Entrando a update_main_plot con aeronave=A3, parametro=Alcance de la aeronave, predictor=__all__
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] triggered_only_selection: False
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] triggered_only_selection: False
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] selected_model_data recibido: None
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] selected_model_data recibido: None
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] selected_model_data: None
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] selected_model_data: None
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] modelos_celda (len=7): ['linear-1', 'log-1', 'pot-1', 'exp-1', 'poly-1', 'log-1', 'pot-1']
INFO:Analisis_modelos.main_visualizacion_modelos:[DEBUG] modelos_celda (len=7): ['linear-1', 'log-1', 'pot-1', 'exp-1', 'poly-1', 'log-1', 'pot-1']
INFO:Analisis_modelos.